In [102]:
import pickle
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.special import binom
from itertools import combinations, permutations

In [121]:
def stirling_sum(Ns):
    """ ...
    """
    stirling = lambda n,k: int(1./math.factorial(k) * np.sum([(-1.)**i * binom(k,i)*(k-i)**n for i in range(k)]))
    return np.sum([stirling(Ns, k) for k in range(Ns+1)])

def partition(S):
    """ ...
    """
    if len(S) == 1:
        yield [S]
        return 

    first = S[0]
    for smaller in partition(S[1:]):
        for n, subset in enumerate(smaller):
            yield smaller[:n]+[[first] + subset]+smaller[n+1:]
        yield [[first]]+smaller 
    
def gen_partitions(S):
    """
    generate all possible partitions of Ns-element set S
    
    Args: 
        S (list): list of non-functional parameters S
    """
    return [p for _, p in enumerate(partition(S),1)]

def gen_combinations(X_funcs, Ng):
    """ generate all possible functional parameter combinations
    given number of non-functional parameter subsets Ng
    
    Args: 
        X_funcs (np.ndarray): numpy array with all functional 
            possile functional parameters
        Ng (int): number of non-functional parameter subsets
    
    Returns
        (np.ndarray): array of parameter combinations of
            shape (# combs, Ng, # params)
    """
    
    return np.array(list(combinations(X_funcs, Ng)))

def gen_permutations(X_funcs, Ng):
    """ generate all possible functional parameter permutations
    given number of non-functional parameter subsets Ng
    
    Args: 
        X_funcs (np.ndarray): numpy array with all functional 
            possile functional parameters
        Ng (int): number of non-functional parameter subsets
        
    Returns
        (np.ndarray): array of parameter permutations of
            shape (# perms, Ng, # params)
    """
    
    return np.array(list(permutations(X_funcs, Ng)))
    
    

In [68]:
S = [1,2,3]
parts = gen_partitions(S)
print(parts)
len(parts)

[[[1, 2, 3]], [[1], [2, 3]], [[1, 2], [3]], [[2], [1, 3]], [[1], [2], [3]]]


5

In [70]:
X_funcs = np.array([[0, 0, 0], [0, 1, 1], [0, 2, 1]])
combs = gen_combinations(X_funcs, 2)
print('num combs : ', combs.shape)
combs

num combs :  (3, 2, 3)


array([[[0, 0, 0],
        [0, 1, 1]],

       [[0, 0, 0],
        [0, 2, 1]],

       [[0, 1, 1],
        [0, 2, 1]]])

In [71]:
X_funcs = np.array([[0, 0, 0], [0, 1, 1], [0, 2, 1]])
perms = gen_permutations(X_funcs, 2)
print('num perms : ', perms.shape)
perms

num perms :  (6, 2, 3)


array([[[0, 0, 0],
        [0, 1, 1]],

       [[0, 0, 0],
        [0, 2, 1]],

       [[0, 1, 1],
        [0, 0, 0]],

       [[0, 1, 1],
        [0, 2, 1]],

       [[0, 2, 1],
        [0, 0, 0]],

       [[0, 2, 1],
        [0, 1, 1]]])

In [45]:
stirling_sum(3)

5

In [72]:
# stirling = lambda n,k: 1./math.factorial(k) * np.sum([(-1.)**i * binom(k,i)*(k-i)**n for i in range(k)])
# n = 15
# ks = range(n+1)
# vals = []
# for k in ks:
#     vals.append(int(stirling(n,k)))
# print(vals)
# print(np.sum(vals))

In [120]:
def record_merits(S, lookup, X_func_truncate=20):
    
    # list of dictionaries to store G, X_func, f_x
    f_xs = [] 
    
    start_time = time.time()
    
    # generate all the partitions of non-functional parameters
    Gs = gen_partitions(S)
    print('total non-functional partitions : ', len(Gs))
    
    # generate all the possible values of functional parametres
    X_funcs = df[
        ['base_ix', 'ligand_ix', 'additive_ix']
    ].drop_duplicates().values
    if isinstance(X_func_truncate,int):
        X_funcs = X_funcs[:X_func_truncate, :]
    print('cardnality of functional params : ', X_funcs.shape[0])
    
    for G_ix, G in enumerate(Gs): 
        if G_ix % 1 == 0:
            print(f'[INFO] Evaluating partition {G_ix+1}/{len(Gs)+1}')
        Ng = len(G)
        # generate permutations of functional params
        X_func_perms = gen_permutations(X_funcs, Ng)
        
        for X_func in X_func_perms:
            # measure objective 
            f_x = measure_objective(X_func, G, lookup)
            # store values
            f_xs.append(
                {
                    'G': G,
                    'X_func': X_func,
                    'f_x': f_x,
                }
            )
    total_time = round(time.time()-start_time,2)
    print(f'[INFO] Done in {total_time} s')
    
    return f_xs


def get_best(f_xs, maximize=False):
    f_x_arr = np.array([d['f_x'] for d in f_xs ])
    if maximize:
        best_ix = np.argmax(f_x_arr)
    else:
        best_ix = np.argmin(f_x_arr)
    return f_xs[best_ix], best_ix

In [123]:
S = list(np.arange(4))
f_xs = record_merits(S, df, X_func_truncate=20)

total non-functional partitions :  15
cardnality of functional params :  20
[INFO] Evaluating partition 1/16
[INFO] Evaluating partition 2/16
[INFO] Evaluating partition 3/16
[INFO] Evaluating partition 4/16
[INFO] Evaluating partition 5/16
[INFO] Evaluating partition 6/16
[INFO] Evaluating partition 7/16
[INFO] Evaluating partition 8/16
[INFO] Evaluating partition 9/16
[INFO] Evaluating partition 10/16
[INFO] Evaluating partition 11/16
[INFO] Evaluating partition 12/16
[INFO] Evaluating partition 13/16
[INFO] Evaluating partition 14/16
[INFO] Evaluating partition 15/16
[INFO] Done in 321.15 s


In [124]:
len(f_xs)

160000

In [118]:
best, best_ix = get_best(f_xs, maximize=True)
print(best_ix)
best

90169


{'G': [[0], [1], [2], [3], [4]],
 'X_func': array([[0, 0, 2],
        [1, 0, 2],
        [2, 0, 1],
        [0, 1, 0],
        [0, 0, 1]]),
 'f_x': 139.677383136}

In [ ]:
fig, ax = plt.subplots()

vals = []
Ngs = [len(d['G']) for d in range()]
f_x_arr = np.array([d['f_x'] for d in f_xs ])

## Buchwald dataset

functional parameters: `base_ix`, `ligand_ix`, `additive_ix`

non-functional/general parameters: `aryl_halide_ix`

objective: `yield` (maximize, percentage)

In [7]:
df = pickle.load(open('buchwald/main_df.pkl', 'rb'))
print(df.shape)
df.head()

(4132, 129)


,base_ix,base_name,ligand_ix,ligand_name,aryl_halide_ix,aryl_halide_name,additive_ix,additive_name,yield,base_*N1_electrostatic_charge,...,additive_E_LUMO,additive_V1_frequency,additive_V1_intensity,additive_dipole_moment,additive_electronegativity,additive_hardness,additive_molecular_volume,additive_molecular_weight,additive_ovality,additive_surface_area
0,0,P2Et,0,XPhos,0,1-chloro-4-(trifluoromethyl)benzene,0,5-phenylisoxazole,10.657812,-0.755,...,-0.0487,906.164,3.681,3.210447,0.14,0.09,154.41,145.161,1.228,170.87
1,0,P2Et,0,XPhos,1,1-bromo-4-(trifluoromethyl)benzene,0,5-phenylisoxazole,14.747896,-0.755,...,-0.0487,906.164,3.681,3.210447,0.14,0.09,154.41,145.161,1.228,170.87
2,0,P2Et,0,XPhos,2,1-iodo-4-(trifluoromethyl)benzene,0,5-phenylisoxazole,18.278686,-0.755,...,-0.0487,906.164,3.681,3.210447,0.14,0.09,154.41,145.161,1.228,170.87
3,0,P2Et,0,XPhos,3,1-chloro-4-methoxybenzene,0,5-phenylisoxazole,2.475058,-0.755,...,-0.0487,906.164,3.681,3.210447,0.14,0.09,154.41,145.161,1.228,170.87
4,0,P2Et,0,XPhos,4,1-bromo-4-methoxybenzene,0,5-phenylisoxazole,6.119058,-0.755,...,-0.0487,906.164,3.681,3.210447,0.14,0.09,154.41,145.161,1.228,170.87


In [1]:
def buchwald_lookup(params, target, lookup):
    """ lookup the yield for a given functional/non-functional
    parameter setting
    """
    base = params[0]
    ligand = params[1]
    additive = params[2]
    sub_df = lookup[
        (lookup['base_ix']==base) &
        (lookup['ligand_ix']==ligand) &
        (lookup['additive_ix']==additive) &
        (lookup['aryl_halide_ix']==target)
    ]
    if sub_df.shape[0] == 1:
        yield_ = sub_df.loc[:, 'yield'].to_numpy()[0]
    elif sub_df.shape[0] < 1:
        yield_ = 0.0
    
    return yield_

def measure_objective(xgs, G, lookup):
    """
    Args:
        x_gs (np.ndarray): array of proposed general parameters (Ng x 1)
        G (dict): keys are indices of non-functional subsets Sg, values are lists
            of non-functional parameter names assigned to that subset
    """
    f_x = 0.
    for g_ix, Sg in enumerate(G):
        for si in Sg:
            f_xg += buchwald_lookup(xgs[g_ix], si, lookup)
        
    return f_x

        

In [11]:
params = [0, 0, 0]
target = 0

yield_ = buchwald_lookup(params, target, df )
print(yield_)

10.65781182


In [75]:
#------------------------------
# solution with Ng = 2 (worse)
#------------------------------
xgs_1 = np.array([
    [0, 0, 0], 
    [0, 0, 1], 
])
G_1 = [
    [0, 2, 4, 6, 8, 10, 12, 14], 
    [1, 3, 5, 7, 9, 11, 13],
]

f_x = measure_objective(xgs_1, G_1, df)
print('f_x : ', f_x )

f_x :  47.50003757621428


In [76]:
#-------------------------------
# solution with Ng = 4 (better) 
#-------------------------------
xgs_2 = np.array([
    [1, 1, 4], 
    [2, 3, 17],
    [2, 1, 1],
    [2, 1, 3]
])
G_2 = [
    [0, 1, 2], 
    [3, 4, 5],
    [6, 7, 8, 9],
    [10, 11, 12, 13, 14],
]

f_x = measure_objective(xgs_2, G_2, df)
print('f_x : ', f_x )

f_x :  257.18961800625


## Suzuki dataset

In [1]:
import numpy as np